<a href="https://colab.research.google.com/github/torn2537/PetSegmentation/blob/master/Animal_segmentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
!pip install -q -U tfds-nightly
!pip install -q git+https://github.com/tensorflow/examples.git
#Import tensorflow and check the version.
import tensorflow as tf
print(tf.__version__)

2.2.0-rc4


In [0]:
from tensorflow_examples.models.pix2pix import pix2pix
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
import matplotlib.pyplot as plt

In [0]:
#Specify a dataset's name and load it with tfds.load()
dataset_name = 'oxford_iiit_pet:3.*.*'
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    dataset_name,
    split=['train[:80%]', 'train[80%:100%]', 'test'],
    with_info=True
    )

In [86]:
print(train)

<ParallelMapDataset shapes: ((128, 128, 3), (128, 128, 1)), types: (tf.float32, tf.float32)>


In [0]:
'''
Create a normalization function which normalizes images to range [0-255].
Moreover, we will 'subtract' masks of images with 1
'''
def normalize(image, mask_image):
  image = tf.cast(image, tf.float32) / 255.0
  mask_image = mask_image - 1
  return image, mask_image

In [0]:
'''
Create a function named 'loadDataForTraining() with one argument: 'data' 
and annotate the function with @tf.function to work with TF.
In the function, we have to resize the data to be a specific size
(Height, Width) For example, resize to be (128,128) and 'augment' the
data with various approaches (flip, rotate, etc.)
'''
@tf.function
def loadDataForTraining(data):
  size = (128,128)

  image = tf.image.resize(data['image'], size)
  mask_image = tf.image.resize(data['segmentation_mask'], size)
  random_seed = 745
  if tf.random.uniform(()) > 0.5:
    image = tf.image.random_flip_left_right(image, random_seed)
    image = tf.image.random_flip_up_down(image, random_seed)
    image = tf.image.random_brightness(image, 0.3, random_seed)
    
    mask_image = tf.image.random_flip_left_right(mask_image, random_seed)
    mask_image = tf.image.random_flip_up_down(mask_image, random_seed)
    mask_image = tf.image.random_brightness(mask_image, 0.3, random_seed)

  image, mask_image = normalize(image, mask_image)
  return image, mask_image

In [0]:
def loadDataForTesting(data):
  size = (128,128)
  image = tf.image.resize(data['image'], size)
  mask_image = tf.image.resize(data['segmentation_mask'], size)
  image, mask_image = normalize(image, mask_image)
  return image, mask_image

In [0]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [0]:
#Split each type of data
train = raw_train.map(loadDataForTraining, num_parallel_calls=AUTOTUNE)
val = raw_validation.map(loadDataForTesting)
test = raw_test.map(loadDataForTesting)